In [0]:
import numpy as np

class DataHelper:
    def __init__(self, data, label):
        self._index_in_epoch = 0
        self._epochs_completed = 0
        self._data = data
        self._label = label
        self._num_examples = data.shape[0]
        pass

    @property
    def data(self):
        return self._data

    @property
    def label(self):
        return self._label

    def next_batch(self, batch_size):
        start = self._index_in_epoch
        if start == 0 and self._epochs_completed == 0:
            idx = np.arange(0, self._num_examples)  # get all possible indexes
            np.random.shuffle(idx)  # shuffle index
            self._data = self.data[idx]  # get list of `num` random samples
            self._label = self.label[idx]

        # go to the next batch
        if start + batch_size > self._num_examples:
            self._epochs_completed += 1
            rest_num_examples = self._num_examples - start
            data_rest_part = self.data[start:self._num_examples]
            label_rest_part = self.label[start:self._num_examples]
            idx0 = np.arange(0, self._num_examples)  # get all possible indexes
            np.random.shuffle(idx0)  # shuffle indexes
            self._data = self.data[idx0]  # get list of `num` random samples
            self._label = self._label[idx0]
            start = 0
            # avoid the case where the #sample != integer times of batch_size
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            data_new_part = self._data[start:end]
            label_new_part = self._label[start:end]
            return np.concatenate((data_rest_part, data_new_part), axis=0), np.concatenate(
                (label_rest_part, label_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self._data[start:end], self._label[start:end]
          


In [0]:


import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255, x_test / 255
# x_train = x_train[:20]
# x_test = x_test[:20]
# y_train = y_train[:20]
# y_test = y_test[:20]

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

data_helper = DataHelper(x_train, y_train)

X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name='image')
Y = tf.placeholder(tf.float32, shape=[None, 10], name='label')
pKeep = tf.placeholder(tf.float32)

K = 6
L = 12
M = 18

W1 = tf.Variable(tf.truncated_normal([5, 5, 3, K], stddev=0.1))
B1 = tf.Variable(tf.ones([K]) / 10)
W2 = tf.Variable(tf.truncated_normal([4, 4, K, L], stddev=0.1))
B2 = tf.Variable(tf.ones([L]) / 10)
W3 = tf.Variable(tf.truncated_normal([3, 3, L, M], stddev=0.1))
B3 = tf.Variable(tf.ones([M]) / 10)

W4 = tf.Variable(tf.truncated_normal([18 * 8 * 8, 200], stddev=0.1))
B4 = tf.Variable(tf.ones([200]) / 10)
W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
B5 = tf.Variable(tf.ones([10]) / 10)

Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME') + B1)
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, 2, 2, 1], padding="SAME") + B2)
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, 2, 2, 1], padding="SAME") + B3)
Y3 = tf.reshape(Y3, [-1, M * 8 * 8])
Y3 = tf.nn.dropout(Y3, pKeep)
Y4 = tf.nn.relu(tf.matmul(Y3, W4) + B4)
y_logit = tf.matmul(Y4, W5) + B5
Y_pred = tf.nn.softmax(y_logit)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_logit, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy)

is_correct = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

train_step = tf.train.GradientDescentOptimizer(0.004).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(25000):
    batch_X, batch_Y = data_helper.next_batch(100)

    train_data = {X: batch_X, Y: batch_Y, pKeep: 0.75}
    sess.run(train_step, feed_dict=train_data)

    acc_train, loss_train = sess.run([accuracy, cross_entropy], feed_dict=train_data)
    test_data = {X: x_test[:1000], Y: y_test[:1000], pKeep: 1.0}
    acc_test, loss_test = sess.run([accuracy, cross_entropy], feed_dict=test_data)

    if i % 100 == 0:
        print(i)
        print("training::: accuracy", acc_train, "loss", loss_train)
        print("testing::: accuracy", acc_test, "loss", loss_test)
        print(50 * '-')


0
training::: accuracy 0.12 loss 2.3178961
testing::: accuracy 0.106 loss 2.3341513
--------------------------------------------------
100
training::: accuracy 0.14 loss 2.2729654
testing::: accuracy 0.174 loss 2.259501
--------------------------------------------------
200
training::: accuracy 0.21 loss 2.2197733
testing::: accuracy 0.173 loss 2.209201
--------------------------------------------------
300
training::: accuracy 0.18 loss 2.1289027
testing::: accuracy 0.2 loss 2.1528516
--------------------------------------------------
400
training::: accuracy 0.2 loss 2.0882955
testing::: accuracy 0.226 loss 2.0963974
--------------------------------------------------
500
training::: accuracy 0.22 loss 2.1009302
testing::: accuracy 0.234 loss 2.071723
--------------------------------------------------
600
training::: accuracy 0.22 loss 2.0844135
testing::: accuracy 0.24 loss 2.0511773
--------------------------------------------------
700
training::: accuracy 0.2 loss 2.0890048
testin